In [1]:
import pandas as pd
import os
import folium
from folium import plugins
import pandas as pd
import folium
from folium import Marker
import geopandas as gpd
import os
import folium.plugins
from IPython.display import display, clear_output
from plotly import express as px
import plotly.graph_objects as go
from shapely.geometry import Point
import numpy as np
import re
from matplotlib import cm
from matplotlib.colors import to_hex
from time import sleep

In [2]:
PATH = os.getcwd()
df = pd.read_csv(f"{PATH}/data/metro.csv", sep=",")
df.head()

,Station,Line,Order of Points,Longitude,Latitude,Traffic
0,Pinar de Chamartin,Linea 1,1,"-3,66706111","40,48013631",6.222.074
1,Bambú,Linea 1,2,"-3,676373739","40,47687195",1.220.848
2,Chamartín,Linea 1,3,"-3,682768126","40,47213836",8.797.197
3,Plaza de Castilla,Linea 1,4,"-3,689162512","40,4668494",25.135.236
4,Valdeacederas,Linea 1,5,"-3,695127745","40,46440066",3.877.235


In [3]:
# Primero arreglamos el formato de los datos
# Las columnas de coordenadas tienen comas en lugar de puntos decimales
df['Longitude'] = df['Longitude'].str.replace(',', '.').astype(float)
df['Latitude'] = df['Latitude'].str.replace(',', '.').astype(float)
# La columna de tráfico tiene puntos que separan miles
df['Traffic'] = df['Traffic'].str.replace('.', '').astype(int)

df.head()

,Station,Line,Order of Points,Longitude,Latitude,Traffic
0,Pinar de Chamartin,Linea 1,1,-3.667061,40.480136,6222074
1,Bambú,Linea 1,2,-3.676374,40.476872,1220848
2,Chamartín,Linea 1,3,-3.682768,40.472138,8797197
3,Plaza de Castilla,Linea 1,4,-3.689163,40.466849,25135236
4,Valdeacederas,Linea 1,5,-3.695128,40.464401,3877235


In [4]:
# Función auxiliar para ordenar las líneas correctamente
def ordenar_lineas(linea):
	return int(linea.replace('Linea ', ''))

In [5]:
def add_tourist_spots(m):
	tourist_spots = [
		{"name": "Museo Reina Sofía", "lat": 40.408735, "lon": -3.694137},
		{"name": "Plaza Mayor", "lat": 40.415365, "lon": -3.707398},
		{"name": "Puerta del Sol", "lat": 40.416775, "lon": -3.703790},
		{"name": "Palacio Real", "lat": 40.417994, "lon": -3.714344},
		{"name": "Museo del Prado", "lat": 40.413782, "lon": -3.692127},
		{"name": "Parque del Retiro", "lat": 40.415260, "lon": -3.684416},
		{"name": "Gran Vía", "lat": 40.420347, "lon": -3.705774},
		{"name": "Templo de Debod", "lat": 40.424021, "lon": -3.717570},
		{"name": "Santiago Bernabéu", "lat": 40.453054, "lon": -3.688344},
		{"name": "Plaza de Cibeles", "lat": 40.419722, "lon": -3.693333},
		{"name": "Mercado de San Miguel", "lat": 40.415363, "lon": -3.708416},
		{"name": "Catedral de la Almudena", "lat": 40.415364, "lon": -3.714451},
		{"name": "El Rastro", "lat": 40.407792, "lon": -3.707177},
		{"name": "Museo Thyssen-Bornemisza", "lat": 40.416873, "lon": -3.694475},
		{"name": "Casa de Campo", "lat": 40.409750, "lon": -3.745571}
	]

	for spot in tourist_spots:
		folium.Marker(
			location=[spot["lat"], spot["lon"]],
			popup=folium.Popup(f"<b>{spot['name']}</b>", max_width=300),
			icon=folium.Icon(color='blue', icon='info-sign')
		).add_to(m)

	folium.LayerControl().add_to(m)

In [6]:
gdf = gpd.read_file(os.path.join(PATH, 'data/neighbourhoods.geojson'))
listings = pd.read_csv(f"{PATH}/data/listings.csv", sep=",")
avg_price_per_neighbourhood = listings.groupby('neighbourhood')['price'].mean().reset_index()
avg_price_per_neighbourhood = avg_price_per_neighbourhood.merge(gdf, left_on='neighbourhood', right_on='neighbourhood')


def add_borders(m):
    folium.GeoJson(
		gdf, 
		name="Distritos de Madrid",
		style_function=lambda feature: {
			'fillColor': 'blue',
			'color': 'black',
			'weight': 1,
			'fillOpacity': 0.1,
		},
		
		tooltip=folium.GeoJsonTooltip(
			fields=['neighbourhood_group'],
			aliases=['Distrito:'],
			localize=True
		)
    
).add_to(m)

In [7]:
def add_cloropleths(m):
    folium.Choropleth(
        geo_data=gdf,
        name='choropleth',
        data=avg_price_per_neighbourhood,
        columns=['neighbourhood', 'price'],
        key_on='feature.properties.neighbourhood',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Precio Promedio (€/noche)',
        highlight=True
    ).add_to(m)

    gdf_with_prices = gdf.merge(avg_price_per_neighbourhood, on="neighbourhood")
    gdf_with_prices["price"] = gdf_with_prices["price"].astype(str).str.replace(',', '.').astype(float)
    gdf_with_prices["price"] = gdf_with_prices["price"].apply(lambda x: f"{x:.2f}")


    gdf_with_prices = gdf_with_prices.rename(columns={'geometry_x': 'geometry'}).set_geometry('geometry')
    if 'geometry_y' in gdf_with_prices.columns:
        gdf_with_prices = gdf_with_prices.drop(columns=['geometry_y'])

    folium.GeoJson(
        data=gdf_with_prices,
        style_function=lambda feature: {
            'fillColor': 'transparent',
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.0,
        },
        popup=folium.GeoJsonPopup(
            fields=['neighbourhood', 'price'],
            aliases=['Distrito:', 'Precio Promedio (€):'],
            localize=True
        )
    ).add_to(m)


In [8]:
# Creamos el mapa con las conexiones
mapa_metro = folium.Map(
	location=[40.4168, -3.7038],
	zoom_start=12
)

add_borders(mapa_metro)

# Colores oficiales del Metro de Madrid
colores_linea = {
	'Linea 1': '#2B7CE9',    # Azul claro
	'Linea 2': '#E6343C',    # Rojo
	'Linea 3': '#FFD700',    # Amarillo
	'Linea 4': '#8B4513',    # Marrón
	'Linea 5': '#4CAF50',    # Verde
	'Linea 6': '#808080',    # Gris
	'Linea 7': '#FF8C00',    # Naranja
	'Linea 8': '#FFC0CB',    # Rosa
	'Linea 9': '#800080',    # Morado
	'Linea 10': '#000080',   # Azul oscuro
	'Linea 11': '#90EE90',   # Verde claro
	'Linea 12': '#DAA520'    # Dorado más oscuro (GoldenRod)
}

# Creamos las capas para cada línea
for linea in sorted(df['Line'].unique(), key=ordenar_lineas):  # Ordenamos usando la función auxiliar
	
	# Creamos un grupo para esta línea
	line_group = folium.FeatureGroup(name=f'{linea}')
	
	# Filtramos y ordenamos las estaciones de esta línea
	df_linea = df[df['Line'] == linea].sort_values('Order of Points')
	
	# Primero dibujamos las conexiones entre estaciones
	coordinates = df_linea[['Latitude', 'Longitude']].values.tolist()
	if len(coordinates) > 1:
		folium.PolyLine(
			locations=coordinates,
			color=colores_linea[linea],
			weight=3,
			opacity=0.8
		).add_to(line_group)
	
	# Calculamos el rango de tráfico para esta línea
	min_traffic = df_linea['Traffic'].min()
	max_traffic = df_linea['Traffic'].max()
	min_radius = 5
	max_radius = 30
	
	# Luego añadimos las estaciones
	for _, row in df_linea.iterrows():
		# Calculamos el radio proporcional al tráfico
		radius = min_radius + (row['Traffic'] - min_traffic) * (max_radius - min_radius) / (max_traffic - min_traffic)
		
		# Añadimos el marcador de la estación
		folium.CircleMarker(
			location=[row['Latitude'], row['Longitude']],
			radius=radius,
			popup=f"Estación: {row['Station']}<br>Línea: {row['Line']}<br>Tráfico: {row['Traffic']:,} pasajeros",
			color=colores_linea[linea],
			fill=True,
			fill_color=colores_linea[linea],
			fill_opacity=0.2,
			weight=3,
			tooltip=f"{row['Station']}: {row['Traffic']:,} pasajeros"
		).add_to(line_group)
	
	line_group.add_to(mapa_metro)

# Añadimos el control de capas
folium.LayerControl(collapsed=False).add_to(mapa_metro)

add_tourist_spots(mapa_metro)
display(mapa_metro)

In [9]:
# Creamos el mapa con las conexiones
mapa_metro = folium.Map(
	location=[40.4168, -3.7038],
	zoom_start=12
)

add_cloropleths(mapa_metro)

# Colores oficiales del Metro de Madrid
colores_linea = {
	'Linea 1': '#2B7CE9',    # Azul claro
	'Linea 2': '#E6343C',    # Rojo
	'Linea 3': '#FFD700',    # Amarillo
	'Linea 4': '#8B4513',    # Marrón
	'Linea 5': '#4CAF50',    # Verde
	'Linea 6': '#808080',    # Gris
	'Linea 7': '#FF8C00',    # Naranja
	'Linea 8': '#FFC0CB',    # Rosa
	'Linea 9': '#800080',    # Morado
	'Linea 10': '#000080',   # Azul oscuro
	'Linea 11': '#90EE90',   # Verde claro
	'Linea 12': '#DAA520'    # Dorado más oscuro (GoldenRod)
}

# Creamos las capas para cada línea
for linea in sorted(df['Line'].unique(), key=ordenar_lineas):  # Ordenamos usando la función auxiliar
	
	# Creamos un grupo para esta línea
	line_group = folium.FeatureGroup(name=f'{linea}')
	
	# Filtramos y ordenamos las estaciones de esta línea
	df_linea = df[df['Line'] == linea].sort_values('Order of Points')
	
	# Primero dibujamos las conexiones entre estaciones
	coordinates = df_linea[['Latitude', 'Longitude']].values.tolist()
	if len(coordinates) > 1:
		folium.PolyLine(
			locations=coordinates,
			color=colores_linea[linea],
			weight=3,
			opacity=0.8
		).add_to(line_group)
	
	# Calculamos el rango de tráfico para esta línea
	min_traffic = df_linea['Traffic'].min()
	max_traffic = df_linea['Traffic'].max()
	min_radius = 5
	max_radius = 30
	
	# Luego añadimos las estaciones
	for _, row in df_linea.iterrows():
		# Calculamos el radio proporcional al tráfico
		radius = min_radius + (row['Traffic'] - min_traffic) * (max_radius - min_radius) / (max_traffic - min_traffic)
		
		# Añadimos el marcador de la estación
		folium.CircleMarker(
			location=[row['Latitude'], row['Longitude']],
			radius=radius,
			popup=f"Estación: {row['Station']}<br>Línea: {row['Line']}<br>Tráfico: {row['Traffic']:,} pasajeros",
			color=colores_linea[linea],
			fill=True,
			fill_color=colores_linea[linea],
			fill_opacity=0.2,
			weight=3,
			tooltip=f"{row['Station']}: {row['Traffic']:,} pasajeros"
		).add_to(line_group)
	
	line_group.add_to(mapa_metro)

# Añadimos el control de capas
folium.LayerControl(collapsed=False).add_to(mapa_metro)

# Mostramos el mapa
add_tourist_spots(mapa_metro)
display(mapa_metro)

In [10]:
# Creamos el mapa con las conexiones
mapa_metro = folium.Map(
	location=[40.4168, -3.7038],
	zoom_start=12
)

# Colores oficiales del Metro de Madrid
colores_linea = {
	'Linea 1': '#2B7CE9',    # Azul claro
	'Linea 2': '#E6343C',    # Rojo
	'Linea 3': '#FFD700',    # Amarillo
	'Linea 4': '#8B4513',    # Marrón
	'Linea 5': '#4CAF50',    # Verde
	'Linea 6': '#808080',    # Gris
	'Linea 7': '#FF8C00',    # Naranja
	'Linea 8': '#FFC0CB',    # Rosa
	'Linea 9': '#800080',    # Morado
	'Linea 10': '#000080',   # Azul oscuro
	'Linea 11': '#90EE90',   # Verde claro
	'Linea 12': '#DAA520'    # Dorado más oscuro (GoldenRod)
}

# Creamos las capas para cada línea
for linea in sorted(df['Line'].unique(), key=ordenar_lineas):  # Ordenamos usando la función auxiliar
	
	# Creamos un grupo para esta línea
	line_group = folium.FeatureGroup(name=f'{linea}')
	
	# Filtramos y ordenamos las estaciones de esta línea
	df_linea = df[df['Line'] == linea].sort_values('Order of Points')
	
	# Primero dibujamos las conexiones entre estaciones
	coordinates = df_linea[['Latitude', 'Longitude']].values.tolist()
	if len(coordinates) > 1:
		folium.PolyLine(
			locations=coordinates,
			color=colores_linea[linea],
			weight=3,
			opacity=0.8
		).add_to(line_group)
	
	# Calculamos el rango de tráfico para esta línea
	min_traffic = df_linea['Traffic'].min()
	max_traffic = df_linea['Traffic'].max()
	min_radius = 5
	max_radius = 30
	
	# Luego añadimos las estaciones
	for _, row in df_linea.iterrows():
		# Calculamos el radio proporcional al tráfico
		radius = min_radius + (row['Traffic'] - min_traffic) * (max_radius - min_radius) / (max_traffic - min_traffic)
		
		# Añadimos el marcador de la estación
		folium.CircleMarker(
			location=[row['Latitude'], row['Longitude']],
			radius=radius,
			popup=f"Estación: {row['Station']}<br>Línea: {row['Line']}<br>Tráfico: {row['Traffic']:,} pasajeros",
			color=colores_linea[linea],
			fill=True,
			fill_color=colores_linea[linea],
			fill_opacity=0.2,
			weight=3,
			tooltip=f"{row['Station']}: {row['Traffic']:,} pasajeros"
		).add_to(line_group)
	
	line_group.add_to(mapa_metro)

# Añadimos el control de capas
folium.LayerControl(collapsed=False).add_to(mapa_metro)

# Calculamos estadísticas por línea
stats_por_linea = df.groupby('Line').agg({
	'Traffic': ['sum', 'mean', 'max', 'min', 'count']
}).round(0)

# Formateamos los números para mejor legibilidad
def format_number(num):
	if num >= 1_000_000:
		return f"{num/1_000_000:.1f}M"
	elif num >= 1_000:
		return f"{num/1_000:.0f}K"
	return str(int(num))

# Creamos el HTML para el panel de estadísticas
stats_html = '''
<div style="position: fixed; 
			bottom: 50px; 
			left: 50px; 
			width: 300px;
			max-height: 500px;
			overflow-y: auto;
			z-index: 1000;
			background-color: white;
			padding: 10px;
			border-radius: 5px;
			border: 2px solid grey;
			font-size: 12px;">
	<p style="margin-bottom: 10px;"><strong>Estadísticas por Línea</strong></p>
	<table style="width: 100%; border-collapse: collapse;">
		<tr style="background-color: #f2f2f2;">
			<th style="padding: 5px; text-align: left; border-bottom: 1px solid #ddd;">Línea</th>
			<th style="padding: 5px; text-align: right; border-bottom: 1px solid #ddd;">Total</th>
			<th style="padding: 5px; text-align: right; border-bottom: 1px solid #ddd;">Promedio</th>
			<th style="padding: 5px; text-align: right; border-bottom: 1px solid #ddd;">Estaciones</th>
		</tr>
'''

# Añadimos cada línea a la tabla
for linea in sorted(df['Line'].unique(), key=lambda x: int(x.replace('Linea ', ''))):
	stats = stats_por_linea.loc[linea]
	stats_html += f'''
		<tr style="border-bottom: 1px solid #ddd;">
			<td style="padding: 5px;">
				<span style="color: {colores_linea[linea]}">●</span> {linea}
			</td>
			<td style="padding: 5px; text-align: right;">{format_number(stats['Traffic']['sum'])}</td>
			<td style="padding: 5px; text-align: right;">{format_number(stats['Traffic']['mean'])}</td>
			<td style="padding: 5px; text-align: right;">{int(stats['Traffic']['count'])}</td>
		</tr>
	'''

stats_html += '''
	</table>
</div>
'''

# Añadimos el panel de estadísticas al mapa
mapa_metro.get_root().html.add_child(folium.Element(stats_html))


# Mostramos el mapa
display(mapa_metro)